### imports

In [3]:
import instructor
from openai import OpenAI
import os
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

True

### load instructor and openai

In [4]:
# Patch the OpenAI client with Instructor
client = instructor.from_openai(OpenAI(api_key=os.environ['OPENAI_API_KEY']))

### user query/proposal idea 💡

In [5]:
user_proposal = input("Please enter the ai app you'd like to develop and the problem you are trying to solve:")

In [6]:
print(user_proposal)

I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to


### BizDev #0 - Build X, get $ in Q1 and $$ in Y1 🤑

In [11]:
prompt_0 = "You are a Business Development Specialist. For a given user input, you are going to search your knowledge base for similar project and project"

class BizDev(BaseModel):
    impact: str = Field(desription="Provide an estimate of how much project revenue this proposal will make in the next quarter as well as in the next year. Use your knowledge base to provide similar examples, provide reasoning and cite your sources.")
    dollars_projected_q1: float = Field(desription="How many dollars (USD) is this project projected to gross in the first quarter (Q1)?")
    dollars_projected_y1: float = Field(desription="How many dollars (USD) is this project projected to gross in the first year (Y1)?")

# Function
def biz_dev_agent(user_proposal) -> BizDev:
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=BizDev,
        messages=[
            {"role": "system", "content": prompt_0},
            {"role": "user", "content": user_proposal},
        ],
    )

In [15]:
biz_dev_output = biz_dev_agent(user_proposal)
biz_dev_output

BizDez(impact='High', dollars_projected_q1=50000.0, dollars_projected_y1=250000.0)

In [18]:
str(biz_dev_output)

"impact='High' dollars_projected_q1=50000.0 dollars_projected_y1=250000.0"

### PRD #1 - Spec Sheet (PRD) Agent with Key Deliverables (What?) 🎛️

In [23]:
prompt_1 = "You are a Spec Sheet (PRD) Agent. For a given proposal, use your knowledge of how to create a Product Requirements Document (PRD) and the sample examples to write one for the proposal."

class PRD(BaseModel):
    prd: str = Field(desription="Write a PRD for this proposal.")

# Function
def prd_agent(biz_dev_output):
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=None,
        messages=[
            {"role": "system", "content": prompt_1},
            {"role": "user", "content": user_proposal + str(biz_dev_output)},
        ],
    )

In [24]:
prd_output = prd_agent(biz_dev_output)
print(prd_output)

ChatCompletion(id='chatcmpl-9mAGL5EGpCWGrLe1dDpvQuLFeSSm3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="# Product Requirements Document (PRD) for a Portfolio Chatbot\n\n## 1. Purpose\nThis document outlines the requirements for a portfolio chatbot designed to showcase the experience and qualifications of an individual for particular roles they may be applying to. The chatbot aims to provide interactive, engaging responses to questions from potential employers or clients, highlighting the individual's accomplishments, skills, and professional background.\n\n## 2. Scope\nThe development, deployment, and maintenance of a chatbot on the individual's professional portfolio website and integration with professional social networks such as LinkedIn. The project targets a reduced response time to inquiries and an improved presentation of the candidate's profile, aiming for increased engagement rates.\n\n## 3. Objectives\n- **Engagement I

In [27]:
print(prd_output.choices[0].message.content)

# Product Requirements Document (PRD) for a Portfolio Chatbot

## 1. Purpose
This document outlines the requirements for a portfolio chatbot designed to showcase the experience and qualifications of an individual for particular roles they may be applying to. The chatbot aims to provide interactive, engaging responses to questions from potential employers or clients, highlighting the individual's accomplishments, skills, and professional background.

## 2. Scope
The development, deployment, and maintenance of a chatbot on the individual's professional portfolio website and integration with professional social networks such as LinkedIn. The project targets a reduced response time to inquiries and an improved presentation of the candidate's profile, aiming for increased engagement rates.

## 3. Objectives
- **Engagement Increase**: Enhance user engagement with the individual’s portfolio by providing immediate, relevant responses.
- **Qualification Highlighting**: Effectively showcase the 

In [28]:
prd_output_content = prd_output.choices[0].message.content

### Designer #2 - (Function) (Engineer) (How?) 

In [32]:
prompt_2 = "You are a the Engineer/Designer Agent a responsible for creating a functional product. For a given proposal and PRD, write the simplest python script to quickly test your PoC. Only output python code and markdown."

# Function
def designer_agent(prd_output_content):
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=None,
        messages=[
            {"role": "system", "content": prompt_1},
            {"role": "user", "content": user_proposal + '\n' + str(biz_dev_output) + '\n' + prd_output_content},
        ],
    )

designer_output = designer_agent(prd_output_content)
print(designer_output)

ChatCompletion(id='chatcmpl-9mBEcMuU8jtfstyzMNZL0a7WRc6WH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The provided document is a comprehensive PRD for a portfolio chatbot, effectively outlining key areas such as the purpose, scope, objectives, features, requirements, milestones, budget, evaluation metrics, dependencies, and risk management. Here are some suggestions and additional points that could further refine and enhance the document:\n\n1. **Detailed Technical Specifications for NLP**:\n   - Specify the necessary languages and frameworks (e.g., TensorFlow, PyTorch) for the Natural Language Processing component.\n   - Include requirements for machine learning model training data, considering privacy and ethical use of data.\n\n2. **User Personalization and Adaptation**:\n   - Detail how the chatbot will adapt over time to provide more personalized responses based on user feedback and interaction patterns.\n\n3. **Integratio

In [33]:
print(designer_output.choices[0].message.content)
# designer_output = designer_output.choices[0].message.content

The provided document is a comprehensive PRD for a portfolio chatbot, effectively outlining key areas such as the purpose, scope, objectives, features, requirements, milestones, budget, evaluation metrics, dependencies, and risk management. Here are some suggestions and additional points that could further refine and enhance the document:

1. **Detailed Technical Specifications for NLP**:
   - Specify the necessary languages and frameworks (e.g., TensorFlow, PyTorch) for the Natural Language Processing component.
   - Include requirements for machine learning model training data, considering privacy and ethical use of data.

2. **User Personalization and Adaptation**:
   - Detail how the chatbot will adapt over time to provide more personalized responses based on user feedback and interaction patterns.

3. **Integration Details with LinkedIn**:
   - Specify the process and technical requirements for integrating the chatbot with LinkedIn, including any needed permissions or API limits t

In [34]:
designer_output = designer_output.choices[0].message.content

### LLM #3 - Assessor (PROs/CONs) (What & How) (pass ✅ or fail ❌ w/ explanation) 

In [36]:
prompt_3 = "You are a the Engineer/Designer Agent a responsible for creating a functional product. For a given proposal and PRD, write the simplest python script to quickly test your PoC. Only output python code and markdown."

class Assessor(BaseModel):
    open_source: bool = Field(desription="Open source")
    cost_of_implementation: float = Field(desription="Cost of implementation in dollars (USD)")
    cost_of_maintainance: float = Field(desription="Cost of maintainance in dollars (USD)")
    partners: str = Field(desription="Partners")
    none_of_the_above: bool = Field(desription="Does this this project fail to address all of these issues?")

# Function
def assessor_agent(designer_output) -> Assessor:
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=Assessor,
        messages=[
            {"role": "system", "content": prompt_1},
            {"role": "user", "content": user_proposal + '\n' + str(biz_dev_output) + '\n' + prd_output_content + '\n' + designer_output},
        ],
    )

assessor_output = assessor_agent(designer_output)
print(assessor_output)

open_source=False cost_of_implementation=50000.0 cost_of_maintainance=250000.0 partners='LinkedIn' none_of_the_above=False


In [41]:
# pass or fail
class PassOrFail(BaseModel):
    pass_or_fail: bool = Field(desription="Pass: True, Fail: False")
    reasoning_for_decision: str = Field(description="Reason why project passed or failed.")

# Function
def pass_or_fail_agent(assessor_output) -> PassOrFail:
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=PassOrFail,
        messages=[
            {"role": "system", "content": "Pass or Fail?"},
            {"role": "user", "content": str(assessor_output)},
        ],
    )

pof_output = pass_or_fail_agent(assessor_output)
print(pof_output)

pass_or_fail=False reasoning_for_decision="The project has a high overall cost, including implementation and maintenance, which may not be justified without clear benefits from the partnership with LinkedIn or open source contributions. The lack of open source contribution limits community support and innovation. It's advisable to re-evaluate the project's objectives, financial model, and potential ROI before proceeding."


In [43]:
if pof_output.pass_or_fail:
    # True
    print("✅ Pass", pof_output.reasoning_for_decision)
else:
    print("❌ Fail", pof_output.reasoning_for_decision)


❌ Fail The project has a high overall cost, including implementation and maintenance, which may not be justified without clear benefits from the partnership with LinkedIn or open source contributions. The lack of open source contribution limits community support and innovation. It's advisable to re-evaluate the project's objectives, financial model, and potential ROI before proceeding.


### PM #4 - Program Manager (Narrative, scrum master)

In [54]:
designer_output_content = designer_output

In [58]:
prompt_4 = "You are a Program Manager. Your tasks include writing narratives, managing scrum processes, and providing clear and concise plans. Consider the cost in terms of time, money, and research for each suggestion. Stack rank them by priority. Do not suggest how to implement them or what tools to use."

class ProgramManager(BaseModel):
    user_prompt: str = Field(description="The initial user proposal.")
    system_prompt: str = Field(description="System prompt describing the role of the Program Manager.")
    response: str = Field(description="The response string containing suggested tasks and reasoning.")

# Function
def program_manager_agent(user_prompt: str, designer_output_content: str): # -> ProgramManager:
    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=None,
        messages=[
            {"role": "system", "content": prompt_4},
            {"role": "user", "content": user_prompt + '\n' + designer_output_content},
        ],
    )
    
    response_content = response.choices[0].message.content.strip()
    print("Response Content:", response_content)
    
    return response_content

# Example usage
pm_output = program_manager_agent(user_proposal, designer_output_content)
print("Program Manager Output:\n", pm_output)

# %%
# print(pm_output.model_dump_json(indent=4))

Response Content: Given the context of creating and refining a portfolio chatbot, the suggestions are prioritized considering the impact on user experience, implementation complexity, and compliance with regulations. Here’s a stack-ranked list based on priority:

1. **Security and Privacy Enhancements**
   - Ensuring user data protection and compliance with global privacy laws like GDPR is crucial for user trust and legal compliance. High priority due to the critical importance of security and privacy in today’s digital environment.

2. **Detailed Technical Specifications for NLP**
   - Defining technical specifications is essential for the development process, guiding the team in choosing the right tools and ensuring the chatbot’s effectiveness in understanding and processing user queries. High priority for foundational development.

3. **Feedback Mechanism for Continuous Improvement**
   - Implementing a structured way to collect user feedback is pivotal for iterative improvements, a